<a href="https://colab.research.google.com/github/imabari/covid19-data/blob/master/ibaraki/ibaraki_covid19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pdfplumber

     |████████████████████████████████| 51kB 3.9MB/s 
     |████████████████████████████████| 5.6MB 8.4MB/s 
     |████████████████████████████████| 143kB 47.4MB/s 
     |████████████████████████████████| 1.9MB 36.9MB/s 
  Created wheel for pdfplumber: filename=pdfplumber-0.5.26-cp36-none-any.whl size=31747 sha256=b27099853b8da2d7ebe4467c1483921161e4a6405bee4b96930579f803d7414b
  Stored in directory: /root/.cache/pip/wheels/5a/4a/09/8f7b594a0d7ed1ca5fe27078a6a96e7439167a502074d87d1b
Successfully built pdfplumber


In [2]:
import datetime
import pathlib
import re
from urllib.parse import urljoin

In [3]:
import requests
from bs4 import BeautifulSoup

In [4]:
import pdfplumber
import pandas as pd

In [5]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko"
}

In [6]:
def fetch_soup(url, parser="html.parser"):

    r = requests.get(url, headers=headers)
    r.raise_for_status()

    soup = BeautifulSoup(r.content, parser)

    return soup

In [7]:
def fetch_file(url, dir="."):

    p = pathlib.Path(dir, pathlib.PurePath(url).name)
    p.parent.mkdir(parents=True, exist_ok=True)
    
    r = requests.get(url)
    r.raise_for_status()

    with p.open(mode="wb") as fw:
        fw.write(r.content)
    return p

In [8]:
def pdf2df(p):

    with pdfplumber.open(p) as pdf:

        dfs = []

        for page in pdf.pages[1:]:

            table = page.extract_table()

            tmp = pd.DataFrame(table[1:], columns=table[0])

            if (tmp.columns[0] == "判明日") or (tmp.columns[1] == "判明日"):

                dfs.append(tmp)

    df = pd.concat(dfs)

    df.replace(["―", "－", ""], pd.NA, inplace=True)
    df.dropna(how="all", inplace=True)

    return df.reset_index(drop=True)

In [9]:
def str2date(s):

    df = s.str.extract("(\d{1,2})月(\d{1,2})日").rename(columns={0: "month", 1: "day"}).fillna(0).astype(int)

    df["year"] = dt_now.year

    tmp = pd.to_datetime(df, errors="coerce")

    df["year"] = df["year"].mask(tmp > dt_now, df["year"] - 1)

    return pd.to_datetime(df, errors="coerce")

In [10]:
JST = datetime.timezone(datetime.timedelta(hours=+9))
dt_now = datetime.datetime.now(JST).replace(tzinfo=None)

In [11]:
url = "https://www.pref.ibaraki.jp/1saigai/2019-ncov/hassei.html"

In [12]:
soup = fetch_soup(url)

# 茨城県

In [13]:
tag_pref = soup.find("a", class_="icon_pdf", text=re.compile("^新型コロナウイルス感染症患者の発生及び退院・退所等について"))
link_pref = urljoin(url, tag_pref.get("href"))

In [14]:
path_pref = fetch_file(link_pref)

In [15]:
df_pref = pdf2df(path_pref)

In [16]:
pub_pref = tag_pref.parent.find_previous_sibling("h4").get_text(strip=True).replace("発表", "")

In [17]:
df_pref["管轄"] = "茨城県"
df_pref["公表日"] = pub_pref

In [18]:
df_pref

,判明日,新規\n濃厚,年代,性別,職業,居住地,発症日,備考（疑われる感染経路）,管轄,公表日
0,2月11日,濃厚,70代,男性,無職,常陸大宮市,2月6日,知人・家庭内,茨城県,2月13日
1,2月11日,濃厚,80代,女性,無職,常陸大宮市,2月6日,知人・家庭内,茨城県,2月13日
2,2月11日,濃厚,70代,女性,非公表,大子町,2月4日,知人・家庭内,茨城県,2月13日
3,2月11日,濃厚,70代,女性,医療従事者,常陸太田市,<NA>,職場内、医療関係,茨城県,2月13日
4,2月11日,濃厚,80代,女性,無職,大洗町,<NA>,知人・家庭内,茨城県,2月13日
5,2月11日,濃厚,50代,女性,会社員,小美玉市,2月9日,職場内,茨城県,2月13日
6,2月12日,新規,40代,男性,会社員,筑西市,2月10日,<NA>,茨城県,2月13日
7,2月12日,新規,30代,男性,会社員,龍ケ崎市,2月7日,<NA>,茨城県,2月13日
8,2月12日,濃厚,50代,女性,会社員,稲敷市,2月11日,知人・家庭内,茨城県,2月13日
9,2月12日,濃厚,30代,男性,パート,取手市,2月11日,知人・家庭内,茨城県,2月13日


# 水戸市

In [19]:
tag_city = soup.find("a", class_="icon_pdf", text=re.compile("^【水戸市発表】新型コロナウイルス感染症患者の発生について"))
link_city = urljoin(url, tag_city.get("href"))

In [20]:
path_city = fetch_file(link_city)

In [21]:
df_city = pdf2df(path_city)

In [22]:
pub_city = tag_city.parent.find_previous_sibling("h4").get_text(strip=True).replace("発表", "")

In [23]:
df_city["管轄"] = "水戸市"
df_city["公表日"] = pub_city

In [24]:
df_city

,例目,判明日,新規\n濃厚,年代,性別,職業,居住地,発症日,管轄,公表日
0,460例目,2月10日,濃厚,10歳代,男子,生徒,茨城町,<NA>,水戸市,2月12日
1,461例目,2月10日,新規,70歳代,女性,パート,水戸市,症状なし,水戸市,2月12日
2,462例目,2月11日,濃厚,50歳代,男性,会社員,水戸市,症状なし,水戸市,2月12日
3,463例目,2月11日,濃厚,40歳代,男性,会社員,水戸市,2月6日,水戸市,2月12日
4,464例目,2月11日,濃厚,40歳代,女性,パート,水戸市,症状なし,水戸市,2月12日
5,465例目,2月11日,濃厚,30歳代,男性,介護職,水戸市,2月6日,水戸市,2月12日


# 結合

In [25]:
df = pd.concat([df_pref, df_city])

In [26]:
df

,判明日,新規\n濃厚,年代,性別,職業,居住地,発症日,備考（疑われる感染経路）,管轄,公表日,例目
0,2月11日,濃厚,70代,男性,無職,常陸大宮市,2月6日,知人・家庭内,茨城県,2月13日,NaN
1,2月11日,濃厚,80代,女性,無職,常陸大宮市,2月6日,知人・家庭内,茨城県,2月13日,NaN
2,2月11日,濃厚,70代,女性,非公表,大子町,2月4日,知人・家庭内,茨城県,2月13日,NaN
3,2月11日,濃厚,70代,女性,医療従事者,常陸太田市,<NA>,職場内、医療関係,茨城県,2月13日,NaN
4,2月11日,濃厚,80代,女性,無職,大洗町,<NA>,知人・家庭内,茨城県,2月13日,NaN
5,2月11日,濃厚,50代,女性,会社員,小美玉市,2月9日,職場内,茨城県,2月13日,NaN
6,2月12日,新規,40代,男性,会社員,筑西市,2月10日,<NA>,茨城県,2月13日,NaN
7,2月12日,新規,30代,男性,会社員,龍ケ崎市,2月7日,<NA>,茨城県,2月13日,NaN
8,2月12日,濃厚,50代,女性,会社員,稲敷市,2月11日,知人・家庭内,茨城県,2月13日,NaN
9,2月12日,濃厚,30代,男性,パート,取手市,2月11日,知人・家庭内,茨城県,2月13日,NaN


# 前処理

In [27]:
# 無症状
df["状態"] = df["発症日"].where(df["発症日"] == "症状なし").replace({"症状なし": "無症状"})

In [28]:
df["年代"] = df["年代"].str.replace("歳代", "代")

In [29]:
df["性別"] = df["性別"].replace({"男子": "男性", "女子": "女性"})

In [30]:
df["患者_濃厚接触者フラグ"] = df["新規\n濃厚"].replace({"新規": 0, "濃厚": 1})

In [31]:
df["判明日"] = str2date(df["判明日"])
df["発症日"] = str2date(df["発症日"])
df["公表日"] = str2date(df["公表日"])

In [32]:
df["判明日ISO"] = df["判明日"].apply(lambda d: pd.Timestamp(d, tz=None).isoformat()).replace("NaT", "")
df["発症日ISO"] = df["発症日"].apply(lambda d: pd.Timestamp(d, tz=None).isoformat()).replace("NaT", "")
df["公表日ISO"] = df["公表日"].apply(lambda d: pd.Timestamp(d, tz=None).isoformat()).replace("NaT", "")

In [33]:
df["全国地方公共団体コード"] = "080004"
df["都道府県名"] = "茨城県"

In [34]:
df["市区町村名"] = df["管轄"].where(df["管轄"] == "水戸市")
df["全国地方公共団体コード"] = df["全国地方公共団体コード"].mask(df["管轄"] == "水戸市", "082015")

In [35]:
df.rename(
    columns={
        "公表日ISO": "公表_年月日",
        "発症日ISO": "発症_年月日",
        "居住地": "患者_居住地",
        "年代": "患者_年代",
        "性別": "患者_性別",
        "職業": "患者_職業",
        "状態": "患者_状態",
        "備考（疑われる感染経路）": "備考",
    },
    inplace=True,
)

In [36]:
df["備考"] = df["備考"].str.replace("、", "感染;") + "感染"

In [37]:
df1 = df.reset_index().sort_values(by=["公表日", "管轄", "index"]).reset_index(drop=True)

In [38]:
df1 = df1.reindex(
    [
        "全国地方公共団体コード",
        "都道府県名",
        "市区町村名",
        "公表_年月日",
        "発症_年月日",
        "患者_居住地",
        "患者_年代",
        "患者_性別",
        "患者_職業",
        "患者_状態",
        "患者_症状",
        "患者_渡航歴の有無フラグ",
        "患者_濃厚接触者フラグ",
        "検査方法",
        "備考",
    ],
    axis=1,
)

In [39]:
df1

,全国地方公共団体コード,都道府県名,市区町村名,公表_年月日,発症_年月日,患者_居住地,患者_年代,患者_性別,患者_職業,患者_状態,患者_症状,患者_渡航歴の有無フラグ,患者_濃厚接触者フラグ,検査方法,備考
0,082015,茨城県,水戸市,2021-02-12T00:00:00,,茨城町,10代,男性,生徒,NaN,NaN,NaN,1,NaN,NaN
1,082015,茨城県,水戸市,2021-02-12T00:00:00,,水戸市,70代,女性,パート,無症状,NaN,NaN,0,NaN,NaN
2,082015,茨城県,水戸市,2021-02-12T00:00:00,,水戸市,50代,男性,会社員,無症状,NaN,NaN,1,NaN,NaN
3,082015,茨城県,水戸市,2021-02-12T00:00:00,2021-02-06T00:00:00,水戸市,40代,男性,会社員,NaN,NaN,NaN,1,NaN,NaN
4,082015,茨城県,水戸市,2021-02-12T00:00:00,,水戸市,40代,女性,パート,無症状,NaN,NaN,1,NaN,NaN
5,082015,茨城県,水戸市,2021-02-12T00:00:00,2021-02-06T00:00:00,水戸市,30代,男性,介護職,NaN,NaN,NaN,1,NaN,NaN
6,080004,茨城県,NaN,2021-02-13T00:00:00,2021-02-06T00:00:00,常陸大宮市,70代,男性,無職,NaN,NaN,NaN,1,NaN,知人・家庭内感染
7,080004,茨城県,NaN,2021-02-13T00:00:00,2021-02-06T00:00:00,常陸大宮市,80代,女性,無職,NaN,NaN,NaN,1,NaN,知人・家庭内感染
8,080004,茨城県,NaN,2021-02-13T00:00:00,2021-02-04T00:00:00,大子町,70代,女性,非公表,NaN,NaN,NaN,1,NaN,知人・家庭内感染
9,080004,茨城県,NaN,2021-02-13T00:00:00,,常陸太田市,70代,女性,医療従事者,NaN,NaN,NaN,1,NaN,職場内感染;医療関係感染


In [40]:
df1.to_csv("080004_ibaraki_covid19_test_people.csv", encoding="utf_8_sig")

In [41]:
df1.to_csv("080004_ibaraki_covid19_test_people.tsv", sep="\t", encoding="utf_8_sig")

In [42]:
from google.colab import files

In [43]:
files.download("080004_ibaraki_covid19_test_people.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>